In [7]:
import sympy as sym
import numpy as np 
from gpkit import ureg as u

from Stability import *


l_h = 7.05 * u.m
h_h = 4.5 * u.m
aircraft_x_cg = 6.32 * u.m
x_np = 0.0 * u.m
wing_b = 13 * u.m
wing_AR = 9
wing_qc = 6.50 * u.m
fuselage_length = 13.5 * u.m
fuselage_width = 2.0 * u.m
wing_c = 1.83 * u.m
wing_taper = 0.32
wing_S = 22 * u.m**2
wing_sweep = 9.1 * u.deg
wing_cl_alpha = 6.4968 * (1/u.rad)
eta_h = 1
S_h = 6.503 * u.m**2
compressibility_correction = 1.31
CL_alpha_h = 4.09 * 1/u.rad
x_ac_h = 14 * u.m

'''
    aircraft_x_cg,
    wing_qc,
    x_aerocenter_tail,
    l_h,
    h_h,
    wing_AR,
    wing_taper,
    wing_c,
    wing_S,
    wing_b,
    wing_sweep,
    wing_cl_alpha,
    fuselage_length,
    fuselage_width,
    eta_h,
    S_h,
    CL_alpha_h,
    compressibility_correction
'''


stab = Stability(
    aircraft_x_cg,
    wing_qc,
    x_ac_h,
    l_h,
    h_h,
    wing_AR,
    wing_taper,
    wing_c,
    wing_S,
    wing_b,
    wing_sweep,
    wing_cl_alpha,
    fuselage_length,
    fuselage_width,
    eta_h,
    S_h,
    CL_alpha_h,
    compressibility_correction
)



print(stab.x_np)
print(stab.SM_Poff)
print(stab.SM_Pon)
print(stab.depsilon_dalpha)
print(stab.dCM_dalpha)

# cruise = []


K_f: 1.6
6.689249411740649 meter
20.177563483095554 dimensionless
21.177563483095554 dimensionless
0.41655092693102835 dimensionless
1.5070211624441143 / radian
